### Librerias Usadas.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import TruncatedSVD, PCA
from gensim.parsing.preprocessing import remove_stopwords
from gensim.models import Word2Vec
from scipy.sparse import csr_matrix, hstack
import tensorflow as tf
import re, string
tf.test.gpu_device_name()

## Lectura de los datos

In [ ]:
dataFake = pd.read_csv("Fake.csv")
dataFake["class"]=0
print("Fake: ",dataFake.shape)

dataTrue = pd.read_csv("True.csv")
dataTrue["class"]=1
print("True: ",dataTrue.shape)

data_merge = pd.concat([dataFake,dataTrue], axis=0)
data = data_merge.drop(["title","subject","date"], axis=1)
print("All data: ",data.shape)

## Limpieza

In [43]:
'''
  Función que elimina los símbolos especiales de un texto,
  así como las stopwords
'''
def word_cleaner(text):
  text = text.lower()
  text = re.sub('\[.*?\]', '', text)
  text = re.sub('\\W', ' ', text)
  text = re.sub('https?://\S+|www\.\S+', '', text)
  text = re.sub('<.*?>+', '', text)
  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  text = re.sub('\n', '', text)
  text = re.sub('\w*\d\w*', '', text)
  return remove_stopwords(text)

'''
  Función que dado un texto, lo limpia y elimina las letras aisladas existentes.
'''
def text_cleaner(text, lemmatizer):
  text = word_cleaner(text)
  tokens = word_tokenize(text)
  lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
  text = remove_letters(lemmatized_tokens)
  text = " ".join(text)
  return text

'''
  Funcion que elimina las palabras con lengitud menor a length
'''
def remove_letters(lemas, length=2):
  return [word for word in lemas if len(word)>length]

'''
  Funcion que cuenta la frecuencia de palabras en el dataset
'''
def count_tokens(texts, wf):
  for text in texts:
    tokens = text.split()
    wf.update(tokens)
  return wf

'''
  Funcion que dado el dataset y la lista de palabras que no tienen una 
  frecuencua valida las elimina del dataset
'''
def remove_max_min_words_freq(texts, words_to_remove):
  filtered_texts = []
  for text in texts:
    tokens = text.split()
    filtered_tokens = [word for word in tokens if word not in words_to_remove]
    filtered_text = ' '.join(filtered_tokens)
    filtered_texts.append(filtered_text)
  return filtered_texts

'''
  Funcion que obtiene la lista de palabras a eliminar
'''
def get_words_to_remove(min_freq =2, max_freq=1000, word_freq=None):
  return [word for word, freq in word_freq.items() if freq < min_freq or freq > max_freq]

### Importaciones para probar la eliminación de palabras con máxima y mínima frecuencia.


In [ ]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter
import nltk
# Descarga de recursos
nltk.download('punkt')
nltk.download('wordnet')
# Instancias 
word_freq = Counter()
lemmatizer = WordNetLemmatizer()

In [8]:
clean_data = data.copy()

In [9]:
clean_data["text"] = clean_data['text'].apply(text_cleaner, args=(lemmatizer,))

### Solo es requerido ejecutar esta celsa si se desea probar la eliminación de palabras con frecuencia max y min.


In [ ]:
head = clean_data.head(100)
tail = clean_data.tail(100)
clean_data = pd.concat([head,tail])
word_freq = count_tokens(clean_data['text'], word_freq)
words = get_words_to_remove(5,1000,word_freq)
clean_data["text"] = remove_max_min_words_freq(clean_data["text"] ,words)

## Separación de datos de entrenamiento y de prueba

In [ ]:
SEED = 123456789

x = clean_data['text']
y = clean_data['class']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=SEED)

print(f"Datos de entrenamiento: {len(x_train)} ({len(x_train)/len(x):%})")
print(f"Datos de prueba: \t{len(x_test)} ({len(x_test)/len(x):%})")

## Preprocesamiento

### Vectorización TFID:

In [45]:
tfid_vectorizer = TfidfVectorizer()
def vectorize_TFID(x, mode="test"):
    if mode == "train":
        return tfid_vectorizer.fit_transform(x)
    elif mode == "test":
        return tfid_vectorizer.transform(x)
    return None

In [50]:
x_tfid_train = vectorize_TFID(x_train, mode="train")
x_tfid_test = vectorize_TFID(x_test, mode="test")

### Vectorización por frecuencia

In [51]:
count_vectorizer = CountVectorizer()
def vectorize_Count(x, mode="test"):
    if mode == "train":
        return count_vectorizer.fit_transform(x)
    elif mode == "test":
        return count_vectorizer.transform(x)
    return None

In [54]:
x_count_train= vectorize_Count(x_train, mode="train")
x_count_test = vectorize_Count(x_test, mode="test")

### Vectorización por Word2Vec


In [47]:
# Vectorización con Word2Vec
def vectorize_W2V(x_train, x_test):
  # Concatenar series2 al frente de series1
  X = pd.concat([x_train, x_test], axis=0)
  sentences = [text.split() for text in X]
  word2vec_model = Word2Vec(sentences, min_count=1)
  # Obtener el tamaño de los vectores de Word2Vec
  embedding_size = word2vec_model.vector_size
  # Obtener el promedio de los vectores de Word2Vec para cada texto
  X_word2vec = np.zeros((len(sentences), embedding_size))
  for i, text in enumerate(sentences):
      count = 0
      for word in text:
          if word in word2vec_model.wv:
              X_word2vec[i] += word2vec_model.wv[word]
              count += 1
      if count > 0:
          X_word2vec[i] /= count
  return X_word2vec

## Reducción de dimensiones con PCA

Dado que KNN es un algoritmo ineficiente para vectores grandes, se generan versiones reducidas en dimensión para cada vectorización.

In [52]:
N_COMPONENTS = 500 # FIXME encontrar mejor valor de N_COMPONENTS

In [53]:
svd_tfid = TruncatedSVD(n_components=N_COMPONENTS)
x_tfid_train_svd = svd_tfid.fit_transform(x_tfid_train)
x_tfid_test_svd = svd_tfid.transform(x_tfid_test)

In [55]:
svd_count = TruncatedSVD(n_components=N_COMPONENTS)
x_count_train_svd = svd_count.fit_transform(x_count_train)
x_count_test_svd = svd_count.transform(x_count_test)

## Ajuste de modelos

### Regresión logística sin SVD

Se ajusta un modelo de regresión logística a los vectores generados por TFID.

In [ ]:
LR_tfid = LogisticRegression(max_iter=1000)
LR_tfid.fit(x_tfid_train, y_train)
print(f"El algoritmo convergió después de {LR_tfid.n_iter_} iteraciones")

Se ajusta un modelo de regresión logística a los vectores generados por frecuencias de palabras.

In [57]:
LR_count = LogisticRegression(max_iter=2000)
LR_count.fit(x_count_train, y_train)
print(f"El algoritmo convergió después de {LR_count.n_iter_} iteraciones")

El algoritmo convergió después de [132] iteraciones


### Regresión Logísitca con SVD

Se ajusta un modelo de regresión logística a los vectores generados por TFID y procesados por SVD.

In [ ]:
LR_tfid_svd = LogisticRegression(max_iter=1000)
LR_tfid_svd.fit(x_tfid_train_svd, y_train)
print(f"El algoritmo convergió después de {LR_tfid_svd.n_iter_} iteraciones")

Se ajusta un modelo de regresión logística a los vectores generados por frecuencias de palabras y procesados por SVD.

In [ ]:
LR_count_svd = LogisticRegression(max_iter=2000)
LR_count_svd.fit(x_count_train_svd, y_train)
print(f"El algoritmo convergió después de {LR_count_svd.n_iter_} iteraciones")

### K Nearest Neighbors con SVD

Se obtiene un valor de $K$ sensato para los modelos.

In [60]:
K = 5 # FIXME Encontrar mejor valor de K
N_JOBS = 4

Se ajusta un modelo KNN a los vectores generados por TFID.

In [ ]:
KNN_tfid_svd = KNeighborsClassifier(n_neighbors=K, n_jobs=N_JOBS)
KNN_tfid_svd.fit(x_tfid_train_svd, y_train)

Se ajusta un modelo KNN a los vectores generados por frecuencias de palabras.

In [ ]:
KNN_count_svd = KNeighborsClassifier(n_neighbors=K, n_jobs=N_JOBS)
KNN_count_svd.fit(x_count_train_svd, y_train)

### Evaluación de los modelos

In [63]:
class TrainedModel:
    def __init__(self, model, name, x_test, y_test) -> None:
        self.model = model
        self.name = name
        self.x_test = x_test
        self.y_test = y_test
        self.predict = model.predict(x_test)

In [64]:
LR_tfid_tm = TrainedModel(LR_tfid, "LR + TFID", x_tfid_test, y_test)

In [65]:
LR_count_tm = TrainedModel(LR_tfid, "LR + Count", x_count_test, y_test)

In [66]:
LR_tfid_svd_tm = TrainedModel(LR_tfid_svd, f"LR + TFID + SVD ({N_COMPONENTS})", x_tfid_test_svd, y_test)

In [67]:
LR_count_svd_tm = TrainedModel(LR_count_svd, f"LR + Count + SVD ({N_COMPONENTS})", x_count_test_svd, y_test)

In [68]:
KNN_tfid_svd_tm = TrainedModel(KNN_tfid_svd, f"KNN ({KNN_tfid_svd.n_neighbors}) + TFID + SVD ({N_COMPONENTS})", x_tfid_test_svd, y_test)

In [69]:
KNN_count_svd_tm = TrainedModel(KNN_count_svd, f"KNN ({KNN_count_svd.n_neighbors}) + Count + SVD ({N_COMPONENTS})", x_count_test_svd, y_test)

In [70]:
trained_models = [
    LR_tfid_tm, LR_tfid_tm, LR_tfid_svd_tm, LR_count_svd_tm, KNN_tfid_svd_tm, KNN_count_svd_tm
]
labels = [0, 1]
target_names = ["Fake", "True"]

### Reportes de clasificación

In [ ]:
def plot_clasification_reports(trained_models, labels, target_names, cols=2):
    # Se generan los reportes
    reports = [
        pd.DataFrame(
            classification_report(
                m.y_test,
                m.predict,
                labels=labels,
                target_names=target_names,
                output_dict=True,
            )
        )
        for m in trained_models
    ]

    # Se obtiene la norma de colores de todos los modelos
    # https://stackoverflow.com/a/70517313/15217078
    values = np.hstack([d.iloc[:-1, :].values.ravel() for d in reports])
    norm = mcolors.Normalize(values.min(), values.max())

    # Se generan las gráficas de los reportes
    # # https://stackoverflow.com/a/58948133/15217078
    rows = int(np.ceil(len(trained_models) / cols))
    fig, axes = plt.subplots(rows, cols)
    fig.set_size_inches(8 * cols, 5 * rows)
    for i, m in enumerate(trained_models):
        ax = axes[i // cols][i % cols]
        r = reports[i]
        sns.heatmap(r.iloc[:-1, :].T, annot=True, norm=norm, ax=ax)
        ax.set_title(m.name)
    return fig, axes


report_fig, report_axes = plot_clasification_reports(
    trained_models, labels, target_names
)


### Composición de vectorizaciones (Word2Vec y CountVectorizer)

In [97]:
# Obtenemos la vectorización de los conjuntos por medio de Count
cv = CountVectorizer()
x_count_train = cv.fit_transform(x_train)
x_count_test = cv.transform(x_test)

In [98]:
# Vectorización con Word2Vec
X_word2vec = vectorize_W2V(x_train, x_test)

In [100]:
# Obtenemos los conjutos ya vectorizados
X_word2vec_train = X_word2vec[:len(x_train)]
X_word2vec_test = X_word2vec[len(x_train):]

In [101]:
# Concatenamos las matrices dispersas y los arreglos a lo largo del eje de las columnas
X_concatenated_train = hstack([x_count_train, X_word2vec_train])
X_concatenated_test = hstack([x_count_test, X_word2vec_test])

In [102]:
# Entrenar el modelo de regresión logística
logreg = LogisticRegression(max_iter=2000)
logreg.fit(X_concatenated_train, y_train)
print(f"El algoritmo convergió después de {logreg.n_iter_} iteraciones")

El algoritmo convergió después de [145] iteraciones


In [103]:
# Predecir las etiquetas para los datos de prueba
y_pred = logreg.predict(X_concatenated_test)

In [104]:
# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print("Precisión del modelo:", accuracy)

Precisión del modelo: 0.9952783964365256


In [105]:
# Reducción de dimensionalidad para X_count
N_COMPONENTS = 100
svd_composition = TruncatedSVD(n_components=N_COMPONENTS)
x_comp_train_svd = svd_composition.fit_transform(x_count_train)
x_comp_test_svd = svd_composition.transform(x_count_test)

In [108]:
# Concatenamos las matrices dispersas y los arreglos a lo largo del eje de las columnas
X_concatenated_train_svd= np.concatenate((x_comp_train_svd, X_word2vec_train), axis=1)
X_concatenated_test_svd= np.concatenate((x_comp_test_svd, X_word2vec_test), axis=1)

In [111]:
# Entrenar el modelo de regresión logística
logreg_svd = LogisticRegression(max_iter=2000)
logreg_svd.fit(X_concatenated_train_svd, y_train)
print(f"El algoritmo convergió después de {logreg_svd.n_iter_} iteraciones")

El algoritmo convergió después de [341] iteraciones


In [112]:
# Predecir las etiquetas para los datos de prueba
y_pred = logreg_svd.predict(X_concatenated_test_svd)

In [ ]:
# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo (SVD sobre Count a {N_COMPONENTS}):", accuracy)

In [ ]:
LR_Word_Count = TrainedModel(logreg, "LR + (Word2Vec & Count)", X_concatenated_test, y_test)
LR_Word_Count_SVD = TrainedModel(logreg_svd, "LR + (Word2Vec & Count & SVD)", X_concatenated_test_svd, y_test)
#tm = [LR_Word_Count, LR_Word_Count_SVD]
trained_models.append(LR_Word_Count)
trained_models.append(LR_Word_Count_SVD)

report_fig, report_axes = plot_clasification_reports(
    trained_models, labels, target_names
)
report_fig, report_axes = plot_clasification_reports(trained_models, labels, target_names)